# インターンレビューデータの分析

## 前準備

### 必要なパッケージ類のインポート

In [1]:
import pandas as pd
import numpy as np
import os
import random
import collections

### 自作関数のインポート

In [2]:
from BertSequenceVectorizer import BertSequenceVectorizer
from my_labspread import my_labspread
from data_mold import data_mold
from my_word2vec import my_word2vec

### データの読み込み

In [3]:
df=pd.read_csv("../data/rev_df.csv")

### テキストデータの分かち書き

In [4]:
word_list=data_mold(df).return_word_list()


### 再現性確保のため乱数を固定

In [5]:
SEED_VALUE = 1
os.environ['PYTHONHASHSEED'] = str(SEED_VALUE)
random.seed(SEED_VALUE)
np.random.seed(SEED_VALUE)

## 業界、企業、業務理解に必要なもの

### word2vecの実行

In [6]:
w2v=my_word2vec()
w2v.fit(word_list)

### 業界理解についての出力

In [7]:
w2v.similar_2words("業界","理解")

 業界 ＋ 理解 に最も近い単語 10 個
SIer 0.9064555764198303
立ち 0.8914197087287903
位置 0.8573115468025208
JSOL 0.8560431599617004
醍醐味 0.8368632197380066
IT 0.8262432813644409
属する 0.8168585896492004
NTT 0.8149785995483398
深める 0.8081704378128052
オービック 0.7754266858100891


### 企業理解についての出力

In [8]:
w2v.similar_2words("企業","理解")

 企業 ＋ 理解 に最も近い単語 10 個
SIer 0.9253706336021423
業界 0.9022322297096252
IT 0.8989577889442444
立ち 0.8799300193786621
NTT 0.8768252730369568
JSOL 0.8669830560684204
位置 0.8665380477905273
醍醐味 0.8393935561180115
通信 0.8382359743118286
属する 0.812191367149353


### 業務理解についての出力

In [9]:
w2v.similar_2words("業務","理解")

 業務 ＋ 理解 に最も近い単語 10 個
SE 0.8442449569702148
SIer 0.8410717248916626
職種 0.8140532970428467
立ち 0.8128588199615479
システムエンジニア 0.7965976595878601
詳しい 0.7952313423156738
位置 0.7888145446777344
醍醐味 0.7776718139648438
体感 0.7647060751914978
知る 0.7615416049957275


## 満足度についての分析

### bertによるテキストデータのベクトル化

In [10]:
BSV = BertSequenceVectorizer()
vec_df = df["rev_text"].progress_apply(lambda x: BSV.vectorize(x))

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 2737/2737 [09:07<00:00,  5.00it/s]


### データの処理(初期のラベルもここで設定)

In [11]:
initial_t_b_list=[845,1294,1391,1429,1751,1956,1964,2123,2442,2645]
initial_t_g_list=[33,68,186,725,850,911,1255,1648,2088,2657]
initial_t=pd.DataFrame({"index":initial_t_g_list+initial_t_b_list,"teacher":list(np.repeat(1,len(initial_t_g_list)))+list(np.repeat(0,len(initial_t_b_list)))})
vec_list = vec_df
teacher_list = [np.nan]*len(vec_list)
for i in range(len(initial_t)):
    teacher_list[initial_t["index"][i]]=initial_t["teacher"][i]

y_data=[i for i in teacher_list]
x_data=[vec_list[i] for i in range(len(teacher_list))]
x_df=pd.DataFrame(x_data)
x_y_df=pd.merge(x_df,pd.DataFrame({"teacher":y_data}),left_index = True,right_index = True)
x_y_df=x_y_df.fillna(-1)
x_data=x_y_df.drop("teacher",axis=1)
y_data=x_y_df["teacher"]

x_data.head()
collections.Counter(y_data)

Counter({-1.0: 2717, 1.0: 10, 0.0: 10})

### ラベル拡張法による半教師あり学習

In [12]:
lbsp=my_labspread()
lbsp.fit(x_data,y_data)
pred_lb=lbsp.return_pre_lab(x_data)

### 導出されたラベルを俯瞰

In [13]:
collections.Counter(pred_lb)

Counter({1.0: 1347, 0.0: 1390})

### ラベルごとにデータを分割

In [14]:
x_py_df=pd.DataFrame({"text":word_list,"teacher":pred_lb})
x_py_vec=pd.DataFrame({"vec":vec_df,"teacher":pred_lb})
x_py_g_df=x_py_df[x_py_df["teacher"]==1]
x_py_b_df=x_py_df[x_py_df["teacher"]==0]
x_py_g_vec=x_py_vec[x_py_df["teacher"]==1]
x_py_b_vec=x_py_vec[x_py_df["teacher"]==0]

### 分割されたデータごとにword2vecの実行

In [15]:
w2v_g=my_word2vec()
w2v_b=my_word2vec()
w2v_g.fit(word_list=x_py_g_df["text"])
w2v_b.fit(word_list=x_py_b_df["text"])

### 出力用の関数を定義

In [16]:
def distance_2words(word1,word2):
    print("満足度が高いレビューにおける「",word1,"」と「",word2,"」の距離：",w2v_g.model.wv.distance(word1,word2))
    print("満足度が低いレビューにおける「",word1,"」と「",word2,"」の距離：",w2v_b.model.wv.distance(word1,word2))

### 高すぎる負荷の部分

In [17]:
distance_2words("内容","つらい")

満足度が高いレビューにおける「 内容 」と「 つらい 」の距離： 0.4205319285392761
満足度が低いレビューにおける「 内容 」と「 つらい 」の距離： 0.667744904756546


### ワークの有無の部分

In [18]:
def counter(word1,word2,li,hl):
    n=0
    for lis in li:
        if word1 in lis or word2 in lis:
            n+=1
    print("満足度が",hl,"レビューでの「",word1,"」、「",word2,"」の出現頻度：",n/len(li))

counter("ワーク","体験",x_py_g_df["text"],"高い")
counter("ワーク","体験",x_py_b_df["text"],"低い")

満足度が 高い レビューでの「 ワーク 」、「 体験 」の出現頻度： 0.7446176688938382
満足度が 低い レビューでの「 ワーク 」、「 体験 」の出現頻度： 0.8129496402877698


### ワークの質の部分

In [19]:
distance_2words("ワーク","退屈")

満足度が高いレビューにおける「 ワーク 」と「 退屈 」の距離： 0.35932815074920654
満足度が低いレビューにおける「 ワーク 」と「 退屈 」の距離： 0.6562124490737915


### コミュニケーションの部分

In [20]:
distance_2words("社員","遠い")
distance_2words("メンバー","遠い")

満足度が高いレビューにおける「 社員 」と「 遠い 」の距離： 0.28223490715026855
満足度が低いレビューにおける「 社員 」と「 遠い 」の距離： 0.46399521827697754
満足度が高いレビューにおける「 メンバー 」と「 遠い 」の距離： 0.21867281198501587
満足度が低いレビューにおける「 メンバー 」と「 遠い 」の距離： 0.43373924493789673
